In [1]:
# Import required libraries
import os, sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import datetime
import matplotlib.colors as mcolors
from scipy.optimize import minimize

# Add parent directory to system path
notebook_dir = os.getcwd()
parent_dir = os.path.dirname(notebook_dir)
sys.path.append(parent_dir)

from erm import RegularizedERM, ERMStabilityEstimator

In [2]:
# Load the COMPAS dataset
df = pd.read_csv('./raw_data/compas-scores-two-years.csv')
df = df[['compas_screening_date', 'sex', 'race', 'v_decile_score', 'is_recid']]
df = df[df.race.isin(["African-American", "Caucasian", "Hispanic"])]
df['phat'] = df['v_decile_score']/10.0
df.compas_screening_date = pd.to_datetime(df.compas_screening_date)
df = df.sort_values(by='compas_screening_date')

# Prepare tensors
y = df.is_recid.to_numpy().astype(float)
yhat = df.phat.to_numpy().astype(float)
# concatenate race and sex dummies
dummy_df_races = pd.get_dummies(df.race)
dummy_df_sexes = pd.get_dummies(df.sex)
dummy_df = pd.concat([dummy_df_races, dummy_df_sexes], axis=1)
races = dummy_df_races.values.astype(float)
sexes = dummy_df_sexes.values.astype(float)
order_races = dummy_df_races.columns.values.tolist()
order_sexes = dummy_df_sexes.columns.values.tolist()

In [3]:
class OLS:
    """
    Ordinary Least Squares regression using RegularizedERM with optional conservative guarantee.
    
    Solves: min_θ (1/2n) ||y - Xθ||² + (λ/2)||θ||² - γ 1_d^T θ
    
    The linear term -γ 1_d^T θ provides a conservative gradient guarantee (Proposition regularized-erm-conservative).
    
    Provides vectorized fit and predict methods.
    """
    
    def __init__(self, lam=0.01, gamma=0.0):
        """
        Initialize OLS regressor.
        
        Parameters
        ----------
        lam : float
            Regularization parameter (ridge penalty)
        gamma : float or np.ndarray
            Linear term coefficient for conservative guarantee.
            If float: same gamma for all coordinates.
            If array: per-coordinate gamma (length d).
        """
        self.lam = lam
        self.gamma = gamma
        self.theta = None
        self.erm = None
    
    def _loss_fn(self, data, theta):
        """
        Squared error loss: (1/2)(y - X^T θ)²
        
        Parameters
        ----------
        data : tuple or array
            If tuple: (X, y) where X is (n, d) and y is (n,)
            If array: assumes structure is data[:, :-1] = X, data[:, -1] = y
        theta : np.ndarray
            Parameter vector (d,)
        
        Returns
        -------
        loss : np.ndarray
            Loss for each sample (n,)
        """
        if isinstance(data, tuple):
            X, y = data
        else:
            X = data[:, :-1]
            y = data[:, -1]
        
        predictions = X @ theta
        return 0.5 * (y - predictions) ** 2
    
    def _grad_fn(self, data, theta):
        """
        Gradient of squared error: -(y - X^T θ) X
        
        Parameters
        ----------
        data : tuple or array
            If tuple: (X, y) where X is (n, d) and y is (n,)
            If array: assumes structure is data[:, :-1] = X, data[:, -1] = y
        theta : np.ndarray
            Parameter vector (d,)
        
        Returns
        -------
        gradient : np.ndarray
            Gradient for each sample (n, d)
        """
        if isinstance(data, tuple):
            X, y = data
        else:
            X = data[:, :-1]
            y = data[:, -1]
        
        predictions = X @ theta
        residuals = y - predictions
        return -(residuals[:, np.newaxis] * X)
    
    def _loss_fn_conservative(self, theta, data):
        """
        Regularized empirical risk with linear term: R̂_D(θ) + (λ/2)||θ||²_2 - γ 1_d^T θ
        """
        return self._loss_fn(data, theta) - np.sum(self.gamma * theta)  # -γ 1_d^T θ
    
    def _gradient_conservative(self, theta, data):
        """
        Gradient with linear term: ∇R̂_D(θ) + λθ - γ 1_d
        """
        return self._grad_fn(data, theta) - self.gamma

    def fit(self, X, y):
        """
        Fit OLS model to training data.
        
        Parameters
        ----------
        X : np.ndarray
            Feature matrix (n, d)
        y : np.ndarray
            Target vector (n,)
        
        Returns
        -------
        self : OLS
            Fitted model
        """
        # Store data as tuple for loss/grad functions
        data = (X, y)
        
        # Initialize theta
        d = X.shape[1]
        theta_init = np.zeros(d)
                
        # If gamma is non-zero, use custom optimization with conservative objective
        self.erm = RegularizedERM(lam=self.lam, theta_init=theta_init)
        self.theta = self.erm.fit(data, self._loss_fn, self._grad_fn)
        
        return self
    
    def predict(self, X):
        """
        Make predictions on new data.
        
        Parameters
        ----------
        X : np.ndarray
            Feature matrix (n, d)
        
        Returns
        -------
        predictions : np.ndarray
            Predicted values (n,)
        """
        if self.theta is None:
            raise ValueError("Model has not been fitted yet. Call fit() first.")
        
        return X @ self.theta


def precompute_beta_ols(X, y, lam=0.01, n_bootstrap=100):
    """
    Precompute stability parameter β for OLS using direct estimation.
    
    Uses ERMStabilityEstimator.estimate_beta_loss_direct() which implements:
    Δ^(b) = (1/(n+1)) Σ_i [ℓ(Z_i^(b); A(D_{-i}^(b))) - ℓ(Z_i^(b); A*(D^(b)))]
    β̂ = E[Δ]_+
    
    Parameters
    ----------
    X : np.ndarray
        Feature matrix (n, d)
    y : np.ndarray
        Target vector (n,)
    lam : float
        Regularization parameter
    n_bootstrap : int
        Number of bootstrap replicates
    
    Returns
    -------
    beta_hat : float
        Estimated stability parameter
    """
    # Create OLS instance to use its loss and gradient functions
    theta_init = np.zeros(X.shape[1])
    ols = OLS(lam=lam, gamma=0.0)
    
    # Prepare data as concatenated array for ERMStabilityEstimator
    data = np.concatenate([X, y[:, np.newaxis]], axis=1)
    
    # Create stability estimator
    estimator = ERMStabilityEstimator(lam=lam, n_bootstrap=n_bootstrap)
    
    # Estimate beta using direct method with OLS loss and gradient functions
    beta_hat = estimator.estimate_beta_loss_direct(
        data=data,
        loss_fn=ols._loss_fn,
        grad_fn=ols._grad_fn,
        theta_init=theta_init
    )
    
    return beta_hat

In [4]:
# Example usage of OLS module
# Create design matrix: intercept + yhat + race dummies + sex dummies
X = np.column_stack([
    np.ones(len(y)),  # intercept
    yhat,             # compas prediction
    races,            # race dummies
    sexes             # sex dummies
])

# Standard OLS (no conservative guarantee)
print("=" * 60)
print("Standard OLS (gamma = 0)")
print("=" * 60)
ols_standard = OLS(lam=0.01, gamma=0.0)
ols_standard.fit(X, y)
y_pred_standard = ols_standard.predict(X)

print(f"Fitted coefficients (θ): {ols_standard.theta}")
print(f"Mean squared error: {np.mean((y - y_pred_standard)**2):.4f}")
print(f"R²: {1 - np.sum((y - y_pred_standard)**2) / np.sum((y - y.mean())**2):.4f}")

# Precompute beta for OLS (this may take a while with bootstrap)
print("\n" + "=" * 60)
print("Precomputing β (stability parameter)")
print("=" * 60)
print("Note: This uses bootstrap with leave-one-out. May take a few minutes...")
beta_hat = precompute_beta_ols(X, y, lam=0.01, n_bootstrap=10)  # Fewer bootstraps for demo
print(f"Estimated β: {beta_hat:.6f}")

# Conservative OLS with gamma = beta
print("\n" + "=" * 60)
print(f"Conservative OLS (gamma = β = {beta_hat:.6f})")
print("=" * 60)
ols_conservative = OLS(lam=0.01, gamma=beta_hat)
ols_conservative.fit(X, y)
y_pred_conservative = ols_conservative.predict(X)

print(f"Fitted coefficients (θ): {ols_conservative.theta}")
print(f"Mean squared error: {np.mean((y - y_pred_conservative)**2):.4f}")
print(f"R²: {1 - np.sum((y - y_pred_conservative)**2) / np.sum((y - y.mean())**2):.4f}")

# Compare predictions
print("\n" + "=" * 60)
print("Comparison")
print("=" * 60)
print(f"Difference in coefficients (L2): {np.linalg.norm(ols_conservative.theta - ols_standard.theta):.6f}")
print(f"Max absolute difference in coefficients: {np.max(np.abs(ols_conservative.theta - ols_standard.theta)):.6f}")

Standard OLS (gamma = 0)
Fitted coefficients (θ): [ 1.42193893e-01  4.71339092e-01  1.01223399e-01  4.13978390e-02
 -4.27345235e-04  2.85793372e-02  1.13614555e-01]
Mean squared error: 0.2260
R²: 0.0954

Precomputing β (stability parameter)
Note: This uses bootstrap with leave-one-out. May take a few minutes...
Estimated β: 0.000121

Conservative OLS (gamma = β = 0.000121)
Fitted coefficients (θ): [ 1.42193893e-01  4.71339092e-01  1.01223399e-01  4.13978390e-02
 -4.27345235e-04  2.85793372e-02  1.13614555e-01]
Mean squared error: 0.2260
R²: 0.0954

Comparison
Difference in coefficients (L2): 0.000000
Max absolute difference in coefficients: 0.000000
